# **Practice 3**

#### **常大伟 03015434**

### **一.问题描述及分析**

* **描述：** 通用Rankine Cycle计算程序设计：能够解析数据文件描述的循环系统拓扑结构，读取热力参数，计算出循环的各项数据，并将结果输出到数据文件。

* **分析：** (1)实现读取数据文件；(2)实现循环中状态点和各项数据的计算；(3)实现循环数据输出到数据文件中。

### **二.设计思路**

#### **1.整体思路**

##### **(1) 类的定义**

11个类——1个状态点类、8个设备类以及两个循环类。

* **状态点类：** 实现利用IF97公式从已知热力参数求未知热力参数的功能，并保存计算结果。

* **设备类：** 实现利用进出设备不同过程（如等熵过程、等压过程、等焓过程等）的特点计算设备涉及到的状态点的计算，并保存计算结果。设备类都包含__init__、state、fdot、simulate、sm_energy、export六个函数。

* **循环类：** RankineCycle和SimRankineCycle。这两类在程序执行过程中说明。
  * RankineCycle：实现对状态点类和设备类的调用和总括，实现循环的计算。
  * SimRankineCycle：通过对RankineCycle类的调用，实现循环的计算和结果的输出。

&nbsp; &nbsp; &nbsp; &nbsp; 下图为状态点类和设备类，设备类以Boiller类为例列举了其函数。

![xmind](./img/xmind2.png)

##### **(2) 程序执行过程**

&nbsp; &nbsp; &nbsp; &nbsp; 在Rankine主函数中新建SimRankineCycle类，SimRankineCycle类新建RankineCycle类，调用CycleSimulator等函数，实现对相应类的实例，完成循环计算，并将结果输出到数据文件中。

![xmind](./img/xmind1.png)

#### **2. 关键模块说明**

##### **(1) 质量流量百分比的计算**

&nbsp; &nbsp; &nbsp; &nbsp; 在Turbine、Closed heater和Open heater三个设备中会出现不同流量工质的分离和合并，因此要计算质量流量百分比相对复杂。

&nbsp; &nbsp; &nbsp; &nbsp; 以Rankine8.6为例，Turbine第一次抽汽y1到Closed heater，第二次抽汽y2到Open heater，同时Closed heater输出y1饱和水经过阀门到Open heater，因此一般思路为先通过Closed heater计算出y1，在通过Open heater计算出y2，最后再计算Turbine的相关点质量流量百分比。

&nbsp; &nbsp; &nbsp; &nbsp; 在设备类的函数中存在fdot函数来计算进出该设备状态点的质量流量，并判断是否进出该设备所有状态点的质量流量百分比都已经求出，没有求出时fdotok为False,并通过循环中的其他设备求解质量流量，直至fdotok为True。

以下为Closed heater计算y1的代码：

    if (self.fdotok == False):
         self.heatAdded = nodes[self.outNode].fdot * \
                    (nodes[self.outNode].h - nodes[self.inNode].h)
        nodes[self.exinNode].fdot = self.heatAdded / \
                    abs (nodes[self.exoutNode].h - nodes[self.exinNode].h)

以下为Open heater计算y2的代码：

    elif ((self.exinNode1 != None)and(self.exinNode2 != None)):
        self.heatAdded = ((nodes[self.outNode].fdot-nodes[self.exinNode1].fdot) *
                        (nodes[self.outNode].h - nodes[self.inNode].h)-
                        nodes[self.exinNode1].fdot * 
                        (nodes[self.exinNode1].h -nodes[self.outNode].h))
        self.heatExtracted = self.heatAdded
        nodes[self.exinNode2].fdot = (self.heatExtracted / 
                        (nodes[self.exinNode2].h - nodes[self.inNode].h))
        nodes[self.inNode].fdot = (nodes[self.outNode].fdot -
                        nodes[self.exinNode1].fdot-nodes[self.exinNode2].fdot)

以下为Turbine计算各状态点质量流量百分比的代码：

    elif ((self.extNode1 != None)and(self.extNode2 != None)):
        nodes[self.reoutNode].fdot = nodes[self.inNode].fdot - nodes[self.extNode1].fdot
        nodes[self.reinNode].fdot = nodes[self.inNode].fdot - nodes[self.extNode1].fdot
        self.fdotok = nodes[self.extNode1].fdot != None 
        if (nodes[self.extNode2].fdot!=None):
            nodes[self.outNode].fdot = nodes[self.inNode].fdot - 
                nodes[self.extNode1].fdot-nodes[self.extNode2].fdot
        self.fdotok = self.fdotok and (nodes[self.extNode2].fdot != None)

##### **(2) 设备数据文件的读取**

&nbsp; &nbsp; &nbsp; &nbsp; 程序通过判断数据文件中的TYPE信息，从而实例化相应的类。以下为实例化没有抽汽的Turbine类的代码。

        if dev[1] == "TURBINE-EX0":
            Comps[dev[0]] = Turbine(dev[0], int(dev[begId]),  int(dev[begId + 1]),
                ef=float(dev[2]))

### **三.小结**

**1.** node类中实现对状态点的计算，设备类实现不同过程中的状态点计算，Rankine类实现对node和设备类的调用，完成循环的计算。

**2.** 在上次作业中，计算吸放热或做功量、耗功量时需要先计算出汽轮机抽汽量y1和y2，没有一般性。在这次练习中,通过设备类中的fdot函数来判断是否进出该设备所有状态点的质量流量百分比都已经求出，没有求出时fdotok为False,并通过循环中的其他设备求解质量流量，直至fdotok为True。这样就不用规定计算的先后顺序，更具有普遍性。

**3.** 设备类中的fdot函数基于能量平衡，并且计算中尽量减少未知量的引入，比如Rankine8.6中的Open heater中计算时nodes[self.inNode].fdot可以用nodes[self.outNode].fdot -nodes[self.exinNode1].fdot-nodes[self.exinNode2].fdot代替。

**4.** 循环的状态点和设备数据文件要保证正确,如设备的进出状态点等。

**5.** 这次的练习实现了通用的循环计算程序，而不是单纯的步骤计算，体会到了普遍性的程序设计难度之大。